<img src="../Pics/MLSb-T.png" width="160">
<br><br>
<center><u><H1>Decision Trees and Random Forest</H1></u></center>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_iris
from sklearn import tree

In [ ]:
iris = load_iris()
dt = tree.DecisionTreeClassifier()
dt = dt.fit(iris.data, iris.target)

In [ ]:
from sklearn.externals.six import StringIO
import pydotplus
from IPython.display import Image  

In [ ]:
import os
import sys

def conda_fix(graph):
        path = os.path.join(sys.base_exec_prefix, "Library", "bin", "graphviz")
        paths = ("dot", "twopi", "neato", "circo", "fdp")
        paths = {p: os.path.join(path, "{}.exe".format(p)) for p in paths}
        graph.set_graphviz_executables(paths)

In [ ]:
buffer = StringIO()
tree.export_graphviz(dt, out_file=buffer, 
                         feature_names=iris.feature_names,  
                         class_names=iris.target_names,  
                         filled=True, rounded=True,  
                         special_characters=True)
graph = pydotplus.graph_from_dot_data(buffer.getvalue())
conda_fix(graph)
graph.write_pdf("iris_tree.pdf") 
Image(graph.create_png())

### References:
http://www.graphviz.org/Download_windows.php

pydotplus install:
conda install -c conda-forge pydotplus=2.0.2

## Predictions:

In [ ]:
iris.data[:1, :]

In [ ]:
dt.predict(iris.data[:1, :])

In [ ]:
dt.predict_proba(iris.data[:1, :])

## Cross Validation:

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(dt, iris.data, iris.target, cv=5)

## Plotting Decision Boundaries:

In [ ]:
def plot_regions(X, y, classifier):
    # Parameters
    n_classes = 3
    plot_colors = "bry"
    res = 0.02

    for pairidx, pair in enumerate([[0, 1], [0, 2], [0, 3],
                                [1, 2], [1, 3], [2, 3]]):
        
        X = iris.data[:, pair]
        # Train
        clf = classifier.fit(X, y)

        # Plot the decision boundary
        fig = plt.figure(1,figsize=(15, 12))
        plt.subplot(2, 3, pairidx + 1)

        x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, res),
                         np.arange(y_min, y_max, res))
        
        output = clf.predict(np.array([xx.ravel(), yy.ravel()]).T)
        output = output.reshape(xx.shape)
        plt.pcolormesh(xx,yy, output, cmap=plt.cm.coolwarm_r)
        
        plt.xlabel(iris.feature_names[pair[0]])
        plt.ylabel(iris.feature_names[pair[1]])
        
        # Plot the training points
        for i, color in zip(range(n_classes), plot_colors):
            idx = np.where(y == i)
            plt.scatter(X[idx, 0], X[idx, 1], c=color, label=iris.target_names[i],
                    cmap=plt.cm.Paired)
          
    plt.suptitle("Decision surface of a decision tree")
    plt.legend(loc='best')
    plt.show()

In [ ]:
# We only take the two corresponding features
X = iris.data
y = iris.target
new_dt = tree.DecisionTreeClassifier()
plot_regions(X=X,y=y,classifier=new_dt)

# Random Forest:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=50)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
# Making predictions on the test set
rf_pred = rf.predict(X_test)
print(confusion_matrix(y_test,rf_pred))

In [ ]:
print(classification_report(y_test,rf_pred))

In [ ]:
cross_val_score(dt, X_test, y_test, cv=5)

## Reference: 

http://scikit-learn.org/stable/modules/tree.html

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html